# 2.4 External Tools

The following section demonstrates how to use the **signac** command line interface (CLI) in conjunction with other tools.

In [1]:
%%bash
signac --help

usage: signac [-h] [--debug] [--version] [-y]
              {init,project,job,statepoint,move,clone,index,find,view,config}
              ...

signac aids in the management, access and analysis of large-scale
computational investigations.

positional arguments:
  {init,project,job,statepoint,move,clone,index,find,view,config}

optional arguments:
  -h, --help            show this help message and exit
  --debug               Show traceback on error for debugging.
  --version             Display the version number and exit.
  -y, --yes             Answer all questions with yes. Useful for scripted
                        interaction.


To interact with a project on the command line, the current working directory needs to be within or below the project root directory.
Let's start by reseting the designated project root directory for this section of the tutorial.

In [2]:
% pwd
% rm -rf projects/tutorial/cli
% mkdir -p projects/tutorial/cli
% cp idg projects/tutorial/cli

Next we switch the current working directory to the project root directory.

In [3]:
% cd projects/tutorial/cli

/home/johndoe/signac-examples/notebooks/projects/tutorial/cli


Then we initialize the project.

In [4]:
%%bash
signac init TutorialCLIProject

Initialized project 'TutorialCLIProject'.


We can verify the project configuration.

In [5]:
%%bash
signac project
signac project --workspace

TutorialCLIProject
/home/johndoe/signac-examples/notebooks/projects/tutorial/cli/workspace


We access the job handle by providing the state point on the command line in JSON format.

In [6]:
%%bash
signac job '{"kT": 1.0, "p": 1.0, "N": 1000}'

ee617ad585a90809947709a7a45dda9a


The statepoints could also be read from STDIN, e.g., by reading it from a file.
Let's create a statepoints file with three statepoints:

In [7]:
%%bash
echo '{"kT": 1.0, "p": 0.1, "N": 1000}' > statepoints.txt
echo '{"kT": 1.0, "p": 1.0, "N": 1000}' >> statepoints.txt
echo '{"kT": 1.0, "p": 10.0, "N": 1000}' >> statepoints.txt

cat statepoints.txt

{"kT": 1.0, "p": 0.1, "N": 1000}
{"kT": 1.0, "p": 1.0, "N": 1000}
{"kT": 1.0, "p": 10.0, "N": 1000}


We can pipe the content of this file into the **signac** CLI to get the corresponding *job id*.

In [8]:
%%bash
head -n 1 statepoints.txt | signac job

Error: expected string or buffer


Instead of the *job id*, we can directly obtain the path to the *job workspace*.

In [9]:
%%bash
head -n 1 statepoints.txt | signac job --workspace

Error: expected string or buffer


That's specifically useful in conjunction with external tools.
Let's pretend that we need to use a program called `idg` to calculate the ideal gas equation.

The `idg` program will calculate the volume of an ideal gas given the input parameters *p*, *kT* and *N*, just like in the previous sections.

In [10]:
%%bash
./idg -p 1.0 -N 1000 --kT 1.0

1000.0


We can store the result in a file based on the input arguments using the `-cw` argument, short for ``--create --workspace``, which returns the workspace path and creates it if necessary.

In [11]:
%%bash
./idg -p 1.0 --kT 1.0 -N 1000 > $(signac job -cw '{"p": 1.0, "kT": 1.0, "N": 1000}')/V.txt

Obviously, we wouldn't write these commands all manually, but use a script instead.

In [12]:
import signac
import json
project = signac.get_project()
for p in 0.1, 1.0, 10.0:
    job = project.open_job({'N': 1000, 'p': p, 'kT': 1.0})
    cmd = './idg -p {p} --kT {kT} -N {N}'.format(**job.statepoint())
    cmd += " > $(signac job -cw '{}')/V.txt".format(json.dumps(job.statepoint()))
    print(cmd)

./idg -p 0.1 --kT 1.0 -N 1000 > $(signac job -cw '{"p": 0.1, "N": 1000, "kT": 1.0}')/V.txt
./idg -p 1.0 --kT 1.0 -N 1000 > $(signac job -cw '{"p": 1.0, "N": 1000, "kT": 1.0}')/V.txt
./idg -p 10.0 --kT 1.0 -N 1000 > $(signac job -cw '{"p": 10.0, "N": 1000, "kT": 1.0}')/V.txt


We can then execute this script...

In [13]:
%%bash
./idg -p 0.1 --kT 1.0 -N 1000 > $(signac job -cw '{"kT": 1.0, "p": 0.1, "N": 1000}')/V.txt
./idg -p 1.0 --kT 1.0 -N 1000 > $(signac job -cw '{"kT": 1.0, "p": 1.0, "N": 1000}')/V.txt
./idg -p 10.0 --kT 1.0 -N 1000 > $(signac job -cw '{"kT": 1.0, "p": 10.0, "N": 1000}')/V.txt

... and examine the results.

In [14]:
% cat `signac project -w`/*/V.txt

100.0
10000.0
1000.0
